# Demonstration of the Available Metrics

For a complete list of metrics and their documentation, please see the API Metrics
[documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook and serves
as an extension of the API documentation to show what the results will look like
depending on what inputs are provided.

In [1]:
from pprint import pprint
from functools import partial

import pandas as pd
from pandas.io.formats.style import Styler

from wombat.core import Simulation, Metrics

# Clean up the aesthetics for the pandas outputs
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 10)
style = partial(
  Styler,
  table_attributes='style="font-size: 14px; grid-column-count: 6"',
  precision=2,
  thousands=",",
)

## Table of Contents

Below is a list of top-level sections to demonstrate how to use WOMBAT's `Metrics`
class methods and an explanation of each individual metric.

If you don't see a metric or result computation that is core to your work, please submit
an [issue](https://github.com/WISDEM/WOMBAT/issues/new) with details on what the metric
is, and how it should be computed.

- [Setup](#setup): Running a simulation to gather the results
- [Common Parameters](#common-parameter-explanations): Explanation of frequently used
  parameter settings
- [Availability](#availability): Time-based and energy-based availability
- [Capacity Factor](#capacity-factor): Gross and net capacity factor
- [Task Completion Rate](#task-completion-rate): Task completion metrics
- [Equipment Costs](#equipment-costs): Cost breakdowns by servicing equipment
- [Service Equipment Utilization Rate](#service-equipment-utilization-rate): Utilization
  of servicing equipment
- [Vessel-Crew Hours at Sea](#vessel-crew-hours-at-sea): Number of crew or vessel hours
  spent at sea
- [Number of Tows](#number-of-tows): Number of tows breakdowns
- [Labor Costs](#labor-costs): Breakdown of labor costs
- [Equipment and Labor Costs](#equipment-and-labor-costs): Combined servicing equipment
  and labor cost breakdown
- [Emissions](#emissions): Emissions of servicing equipment based on activity
- [Component Costs](#component-costs): Materials costs
- [Fixed Cost Impacts](#fixed-cost-impacts): Total fixed costs
- [OpEx](#opex): Project OpEx
- [Process Times](#process-times): Timing of various stages of repair and maintenance
- [Power Production](#power-production): Potential and actually produced power
- [Net Present Value](#net-present-value): Project NPV calculator

(setup)=
## Setup

The simulations from the *How To* notebook are going to be rerun as it is not
recommended to create a Metrics class from scratch due to the large number of inputs
that are required, and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs
required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files()

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

## Common Parameter Explanations

Before diving into each and every metric, and how they can be customized, it is worth
noting some of the most common parameters used throughout, and their meanings to reduce
redundancy. The varying output forms are demonstrated in the
[availability](#availability) section below.

### `frequency`

  project
  : Computed across the whole simulation, with the resulting `DataFrame` having an empty
    index.

  annual
  : Summary of each year in the simulation, with the resulting `DataFrame` having "year"
    as the index.

  monthly
  : Summary of each month of the year, aggregated across years, with the resulting
    `DataFrame` having "month" as the index.

  month-year
  : computed on a month-by-year basis, producing the results for every month of the
    simulation, with the resulting `DataFrame` having "year" and "month" as the index.

### `by`

  windfarm
  : Aggregated across all turbines, with the resulting `DataFrame` having only
    "windfarm" as a column

  turbine
  : Computed for each turbine, with the resulting `DataFrame` having a column for each
    turbine

## Availability

There are two methods to produce availability, which have their own function calls:

- energy: actual power produced divided by potential power produced
  - {py:meth}`wombat.core.post_processor.Metrics.production_based_availability`
- time: The ratio of all non-zero hours to all hours in the simulation, or the
  proportion of the simulation where turbines are operating
  - {py:meth}`wombat.core.post_processor.Metrics.time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations) options: "project",
  "annual", "monthly", and "month-year"
- `by`, as explained [above](#common-parameter-explanations) options: "windfarm" and
  "turbine"

Below is a demonstration of the variations on `frequency` and `by` for
`time_based_availability`.

In [3]:
style(metrics.time_based_availability(frequency="project", by="windfarm"))

,windfarm
0,1.00


In [4]:
style(metrics.production_based_availability(frequency="project", by="windfarm"))

,windfarm
0,0.97


Note that in the two above examples, that the values are equal. This is due to the fact
that the example simulation does not have any operating reduction applied to failures,
unless it's a catastrophic failure, so there is no expected difference.

In [5]:
# Demonstrate the by turbine granularity
style(metrics.time_based_availability(frequency="project", by="turbine"))

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.97,0.97,0.97,0.95,0.98,0.97,0.98,0.96,0.96,0.98,0.98,0.94,0.97,0.96,0.98,0.94,0.97,0.96,0.96,0.96,0.98,0.97,0.97,0.93,0.98,0.97,0.98,0.95,0.90,0.97,0.96,0.95,0.97,0.94,0.96,0.94,0.96,0.95,0.97,0.97,0.98,0.98,0.97,0.98,0.97,0.98,0.98,0.98,0.97,0.96,0.95,0.98,0.95,0.97,0.97,0.97,0.95,0.97,0.97,0.97,0.98,0.94,0.97,0.97,0.98,0.95,0.97,0.98,0.95,0.97,0.97,0.98,0.97,0.97,0.97,0.97,0.98,0.98,0.96,0.98


In [6]:
# Demonstrate the annualized outputs
style(metrics.time_based_availability(frequency="annual", by="windfarm"))

,windfarm
year,
2003,1.00
2004,1.00
2005,1.00
2006,1.00
2007,1.00
2008,1.00
2009,1.00
2010,1.00
2011,1.00


In [7]:
# Demonstrate the month aggregations
style(metrics.time_based_availability(frequency="monthly", by="windfarm"))

,windfarm
month,
1,1.00
2,1.00
3,1.00
4,1.00
5,1.00
6,1.00
7,1.00
8,1.00
9,1.00


In [8]:
# Demonstrate the granular monthly reporting
style(metrics.time_based_availability(frequency="month-year", by="windfarm"))

## Capacity Factor

The capacity factor is the ratio of actual (net) or potential (gross) energy production
divided by the project's capacity. For further documentation, see the API docs here:
{py:meth}`wombat.core.post_processor.Metrics.capacity_factor`.

**Inputs**:

- `which`
  - "net": net capacity factor, actual production divided by the plant capacity
  - "gross": gross capacity factor, potential production divided by the plant capacity
- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"
- `by`, as explained [above](#common-parameter-explanations), options: "windfarm" and
  "turbine"

**Example Usage**:

In [9]:
net_cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
gross_cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
print(f"  Net capacity factor: {net_cf:.2%}")
print(f"Gross capacity factor: {gross_cf:.2%}")

  Net capacity factor: 46.04%
Gross capacity factor: 47.67%


## Task Completion Rate

The task completion rate is the ratio of tasks completed aggregated to the desired
`frequency`. It is possible to have a >100% completion rate if all maintenance and
failure requests submitted in a time period were completed in addition to those that
went unfinished in prior time periods. For further documentation, see the API docs here:
{py:meth}`wombat.core.post_processor.Metrics.task_completion_rate`.

**Inputs**:

- `which`
  - "scheduled": scheduled maintenance only (classified as maintenance tasks in inputs)
  - "unscheduled": unscheduled maintenance only (classified as failure events in inputs)
  - "both": Combined completion rate for all tasks
- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"

**Example Usage**:

In [10]:
scheduled = metrics.task_completion_rate(which="scheduled", frequency="project").values[0][0]
unscheduled = metrics.task_completion_rate(which="unscheduled", frequency="project").values[0][0]
combined = metrics.task_completion_rate(which="both", frequency="project").values[0][0]
print(f"  Scheduled Task Completion Rate: {scheduled:.2%}")
print(f"Unscheduled Task Completion Rate: {unscheduled:.2%}")
print(f"    Overall Task Completion Rate: {combined:.2%}")

  Scheduled Task Completion Rate: 67.93%
Unscheduled Task Completion Rate: 99.59%
    Overall Task Completion Rate: 97.15%


## Equipment Costs

Sum of the costs associated with a simulation's servicing equipment, which excludes
materials, downtime, etc. For further documentation, see the API docs here:
{py:meth}`wombat.core.post_processor.Metrics.equipment_costs`.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"
- `by_equipment`
  - `True`: Aggregates all equipment into a single cost
  - `False`: Computes for each unit of servicing equipment

**Example Usage**:

In [11]:
# Project total at the whole wind farm level
style(metrics.equipment_costs(frequency="project", by_equipment=False))

,equipment_cost
0,"120,083,490.99"


In [12]:
# Project totals at servicing equipment level
style(metrics.equipment_costs(frequency="project", by_equipment=True))

/Users/rhammond/GitHub_Public/WOMBAT/wombat/core/post_processor.py:740: FutureWarning: <class 'pandas.core.arrays.string_.StringArray'>._reduce will require a `keepdims` parameter in the future
  costs = costs.fillna(costs.max(axis=0)).T


,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,390,493.19","6,390,489.48","6,390,488.37","3,003,573.77","97,908,446.18"


## Service Equipment Utilization Rate

Ratio of days when the servicing equipment is in use (not delayed for a whole day due to
either weather or lack of repairs to be completed) to the number of days it's present in
the simulation. For further documentation, see the API docs here:
{py:meth}`wombat.core.post_processor.Metrics.service_equipment_utilization`.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations), options: "project"
  and "annual"

**Example Usage**:

In [13]:
# Project totals
style(metrics.service_equipment_utilization(frequency="project"))

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.93,0.93


## Vessel-Crew Hours at Sea

The number of vessel hours or crew hours at sea for offshore wind power plant
simulations. For further documentation, see the API docs here:
{py:meth}`wombat.core.post_processor.Metrics.vessel_crew_hours_at_sea`.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"
- `by_equipment`
  - `True`: Aggregates all equipment into a single cost
  - `False`: Computes for each unit of servicing equipment
- `vessel_crew_assumption`: A dictionary of vessel names
  (`ServiceEquipment.settings.name`, but also found at `Metrics.service_equipment_names`)
  and the number of crew onboard at any given time. The application of this assumption
  transforms the results from vessel hours at sea to crew hours at sea.

**Example Usage**:

In [14]:
# Project total, not broken out by vessel
style(metrics.vessel_crew_hours_at_sea(frequency="project", by_equipment=False))

,Total Crew Hours at Sea
0,"162,196.73"


In [15]:
# Annual project totals, broken out by vessel
style(metrics.vessel_crew_hours_at_sea(frequency="annual", by_equipment=True))

,Total Crew Hours at Sea,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Heavy Lift Vessel,Field Support Vessel
year,,,,,,
2003,"9,398.61","1,929.74","1,605.72","1,845.99","3,381.15",636.00
2004,"15,544.63","4,224.98","4,233.19","4,206.22","2,279.25",601.00
2005,"17,115.65","4,140.65","4,089.64","4,117.68","4,125.17",642.50
2006,"16,844.71","4,198.70","4,206.49","4,224.73","3,609.80",605.00
2007,"14,668.14","4,170.52","4,161.46","4,150.00","2,186.16",0.00
2008,"18,268.43","4,224.47","4,230.25","4,233.40","3,773.16","1,807.15"
2009,"16,236.55","4,291.98","4,302.98","4,292.26","3,254.98",94.35
2010,"17,340.84","4,157.97","4,155.50","4,181.73","4,199.63",646.00
2011,"16,723.50","4,147.50","4,162.24","4,147.98","3,633.78",632.00


## Number of Tows

The number of tows performed during the simulation. If tow-to-port was not used in the
simulation, a DataFrame with a single value of 0 will be returned. For further
documentation, see the API docs here:
{py:meth}`wombat.core.post_processor.Metrics.number_of_tows`.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"
- `by_tug`
  - `True`: Computed for each tugboat (towing vessel)
  - `False`: Aggregates all the tugboats
- `by_direction`
  - `True`: Computed for each direction a tow was performed (to port or to site)
  - `False`: Aggregates to the total number of tows

**Example Usage**:

In [16]:
# Project Total
# NOTE: This example has no towing, so it will return 0
style(metrics.number_of_tows(frequency="project"))

,total_tows
0,0


## Labor Costs

Sum of all labor costs associated with servicing equipment, excluding the labor defined
in the fixed costs, which can be broken out by type. For further documentation, see the
API docs here: {py:meth}`wombat.core.post_processor.Metrics.labor_costs`.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"
- `by_type`
  - `True`: Computed for each labor type (salary and hourly)
  - `False`: Aggregates all the labor costs

**Example Usage**:

In [17]:
# Project total at the whole wind farm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [18]:
# Project totals for each type of labor
# NOTE: this simulation relies on using a fixed labor cost, so this is still $0
style(metrics.labor_costs(frequency="project", by_type=True))

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0,0,0


## Equipment and Labor Costs

Sum of all labor and servicing equipment costs, excluding the labor defined in the fixed
costs, which can be broken out by each category. For further documentation, see the API
docs here: {py:meth}`wombat.core.post_processor.Metrics.equipment_labor_cost_breakdown`.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"
- `by_category`
  - `True`: Computed for each unit servicing equipment and labor category
  - `False`: Aggregated to the sum of all costs

`reason` definitions:

- Maintenance: routine maintenance, or events defined as a
  {py:class}`wombat.core.data_classes.Maintenance`
- Repair: unscheduled maintenance, ranging from inspections to replacements, or events
  defined as a {py:class}`wombat.core.data_classes.Failure`
- Mobilization: Cost of mobilizing servicing equipment
- Crew Transfer: Costs incurred while crew are transferring between a turbine or
  substation and the servicing equipment
- Site Travel: Costs incurred while transiting to/from the site and while at the site
- Weather Delay: Any delays caused by unsafe weather conditions
- No Requests: Equipment and labor is active, but there are no repairs or maintenance
  tasks to be completed
- Not in Shift: Any time outside the operating hours of the wind farm (or the
  servicing equipment's specific operating hours)

**Example Usage**:

In [19]:
# Project totals
style(metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False))

,total_cost,total_hours
reason,,
Maintenance,"2,100,000.00","28,800.00"
Repair,"35,540,194.70","64,547.94"
Crew Transfer,"1,184,078.12","10,431.00"
Site Travel,0.00,0.00
Mobilization,"9,500,000.00","33,408.00"
Weather Delay,"30,414,050.21","35,754.15"
No Requests,"32,320,462.93","28,012.52"
Not in Shift,"9,024,705.02","117,203.75"


In [20]:
# Project totals by each category
style(metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True))

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost,total_hours
reason,,,,,,
Maintenance,0,0,0,"2,100,000.00","2,100,000.00","28,800.00"
Repair,0,0,0,"35,540,194.70","35,540,194.70","64,547.94"
Crew Transfer,0,0,0,"1,184,078.12","1,184,078.12","10,431.00"
Site Travel,0,0,0,0.00,0.00,0.00
Mobilization,0,0,0,"9,500,000.00","9,500,000.00","33,408.00"
Weather Delay,0,0,0,"30,414,050.21","30,414,050.21","35,754.15"
No Requests,0,0,0,"32,320,462.93","32,320,462.93","28,012.52"
Not in Shift,0,0,0,"9,024,705.02","9,024,705.02","117,203.75"


## Emissions

Emissions (tons or other provided units) of all servicing equipment activity, except
overnight waiting periods between shifts. For further documentation, see the API docs
here: {py:meth}`wombat.core.post_processor.Metrics.emissions`.

**Inputs**:

- `emissions_factors`: Dictionary of servicing equipment names and the emissions per
  hour of the following activities: `transit`, `maneuvering`, `idle at site`, and
  `idle at port`, where port is stand-in for wherever the servicing equipment might be
  based when not at site.
- `maneuvering_factor`: The proportion of transit time that can generally be associated
  with positioning servicing, by default 10%.
- `port_engine_on_factor`: The proportion of the idling at port time when the engine is
  running and producing emissions, by default 25%.

In [21]:
# Create the emissions factors, in tons per hour
emissions_factors = {
    "Crew Transfer Vessel 1": {
        "transit": 4,
        "maneuvering": 3,
        "idle at site": 0.5,
        "idle at port": 0.25,
    },
    "Crew Transfer Vessel 2": {
        "transit": 4,
        "maneuvering": 3,
        "idle at site": 0.5,
        "idle at port": 0.25,
    },
    "Crew Transfer Vessel 3": {
        "transit": 4,
        "maneuvering": 3,
        "idle at site": 0.5,
        "idle at port": 0.25,
    },
    "Field Support Vessel": {
        "transit": 6,
        "maneuvering": 4,
        "idle at site": 1,
        "idle at port": 0.5,
    },
    "Heavy Lift Vessel": {
        "transit": 12,
        "maneuvering": 7,
        "idle at site": 1,
        "idle at port": 0.5,
    },
}

style(metrics.emissions(emissions_factors=emissions_factors, maneuvering_factor=0.075, port_engine_on_factor=0.20))

## Component Costs

All the costs associated with maintenance and failure events during the simulation,
including delays incurred during the repair process, but excluding costs not directly
tied to a repair. For further documentation, see the API docs here:
{py:meth}`wombat.core.post_processor.Metrics.component_costs`.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"
- `by_category`
  - `True`: Computed across each cost category
  - `False`: Aggregated to the sum of all categories
- `by_action`
  - `True`: Computed by each of "repair", "maintenance", and "delay", and is included in
    the MultiIndex
  - `False`: Aggregated as the sum of all actions

`action` definitions:

- maintenance: routine maintenance
- repair: unscheduled maintenance, ranging from inspections to replacements
- delay: Any delays caused by unsafe weather conditions or not being able to finish a
  process within a single shift

**Example Usage**:

In [22]:
# Project totals by component
style(metrics.component_costs(frequency="project", by_category=False, by_action=False))

,total_cost
component,
turbine,"130,577,212.07"


In [23]:
# Project totals by each category and action type
style(metrics.component_costs(frequency="project", by_category=True, by_action=True))

## Fixed Cost Impacts

Computes the total costs of the fixed costs categories. For further documentation, see
the definition docs, here: {py:class}`wombat.core.data_classes.FixedCosts`, or the API
docs here: {py:meth}`wombat.core.post_processor.Metrics.fixed_costs`.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"
- `resolution` (also, demonstrated below)
  - "high": Computed across the most granular cost levels
  - "medium": Computed for each general cost category
  - "low": Aggregated to a single sum of costs

In [24]:
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


**Example Usage**:

In [25]:
# Project totals at the highest level
# NOTE: there were no fixed costs defined in this example, so all values will be 0, so
#       this will just be demonstrating the output format
style(metrics.project_fixed_costs(frequency="project", resolution="low"))

,operations
0,"16,013,230.75"


In [26]:
# Project totals at the medium level
style(metrics.project_fixed_costs(frequency="project", resolution="medium"))

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [27]:
# Project totals at the lowest level
style(metrics.project_fixed_costs(frequency="project", resolution="high"))

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


## OpEx

Computes the total cost of all operating expenditures for the duration of the
simulation, including fixed costs. For further documentation, see the API docs here:
{py:meth}`wombat.core.post_processor.Metrics.opex`.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"

- `by_category`
  - `True` shows the port fees, fixed costs, labor costs, equipment costs, and materials
    costs in addition the total OpEx
  - `False` shows only the total OpEx

**Example Usage**:

In [28]:
style(metrics.opex("annual"))

,OpEx
year,
2003,"11,336,836.24"
2004,"18,936,353.15"
2005,"20,621,343.14"
2006,"17,078,727.06"
2007,"15,004,115.44"
2008,"19,815,276.60"
2009,"13,014,482.40"
2010,"26,361,501.58"
2011,"19,194,277.86"


In [29]:
style(metrics.opex("annual", by_category=True))

,operations,port_fees,equipment_cost,total_labor_cost,materials_cost,OpEx
year,,,,,,
2003,"1,600,008.00",0,"7,192,328.24",0,"2,544,500","11,336,836.24"
2004,"1,604,391.58",0,"12,333,461.56",0,"4,998,500","18,936,353.15"
2005,"1,600,008.00",0,"13,129,835.14",0,"5,891,500","20,621,343.14"
2006,"1,600,008.00",0,"10,378,219.06",0,"5,100,500","17,078,727.06"
2007,"1,600,008.00",0,"9,708,607.44",0,"3,695,500","15,004,115.44"
2008,"1,604,391.58",0,"12,667,885.01",0,"5,543,000","19,815,276.60"
2009,"1,600,008.00",0,"7,246,974.40",0,"4,167,500","13,014,482.40"
2010,"1,600,008.00",0,"17,409,993.58",0,"7,351,500","26,361,501.58"
2011,"1,600,008.00",0,"12,392,769.86",0,"5,201,500","19,194,277.86"


## Process Times

Computes the total number of hours spent from repair request submission to completion,
performing repairs, and the number of request for each repair category. For further
documentation, see the API docs here:
{py:meth}`wombat.core.post_processor.Metrics.process_times`.

**Example Usage**:

In [30]:
style(metrics.process_times())

,time_to_completion,process_time,downtime,time_to_start,N
category,,,,,
annual service,"5,956,041.46","76,421.70","125,422.98","4,205,532.09","1,054"
major repair,"81,213.04","2,712.84","7,465.76","72,581.37",45
major replacement,"176,481.25","9,497.93","14,829.95","166,417.23",96
manual reset,"1,307,146.63","83,714.56","95,690.00","1,206,095.15","8,655"
medium repair,"24,831.65","18,009.88","17,895.23","7,198.78",310
minor repair,"199,213.83","66,157.45","66,341.48","122,615.55","3,519"


## Power Production

Computes the total power production for the wind farm. For further documentation, see
the API docs here: {py:meth}`wombat.core.post_processor.Metrics.power_production`.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"
- `by`, as explained [above](#common-parameter-explanations) options: "windfarm" and "turbine"
- `units`
  - "kwh": kilowatt-hours (kWh)
  - "mwh": megawatt-hours (MWh)
  - "gwh": gigawatt-hours (GWh)

**Example Usage**:

In [31]:
# Project totals, in kWh, at the wind farm level
style(metrics.power_production(frequency="project", by="windfarm", units="kwh"))

,windfarm
Project Energy Production (kWh),"9,687,555,774.50"


In [32]:
# Project totals, in MWh, at the wind farm level
style(metrics.power_production(frequency="project", units="mwh"))

,windfarm
Project Energy Production (MWh),"9,687,555.77"


In [33]:
# Project totals, in GWh, at the wind farm level
style(metrics.power_production(frequency="project"))

,windfarm
Project Energy Production (GWh),"9,687.56"


## Net Present Value

Calcualtes the net present value (NPV) for the project, as
$NPV = (Power * OfftakePrice - OpEx) / (1 + DiscountRate)$.

For further documentation, see the API docs here: {py:meth}`wombat.core.post_processor.Metrics.npv`.

**Inputs**:

- `frequency`, as explained [above](#common-parameter-explanations), options: "project",
  "annual", "monthly", and "month-year"
- `discount_rate`: The rate of return that could be earned on alternative investments,
  by default 0.025.
- `offtake_price`: Price of energy, per MWh, by default 80.

In [34]:
style(metrics.opex("annual"))

,OpEx
year,
2003,"11,336,836.24"
2004,"18,936,353.15"
2005,"20,621,343.14"
2006,"17,078,727.06"
2007,"15,004,115.44"
2008,"19,815,276.60"
2009,"13,014,482.40"
2010,"26,361,501.58"
2011,"19,194,277.86"
